In [1]:
from constants import img_size
import numpy as np
from keras import backend as K
from keras.layers import Input, Dense, subtract
from keras.models import Model
from keras.applications.resnet50 import ResNet50
import pandas as pd
import numpy as np

Using TensorFlow backend.


## Model

In [212]:
class Seamese_Model:
    def __init__(self, base_model, opts, kernel_opts=None, bias_opts=None):
        self.base_model = base_model
        self.opts = opts
        self.kernel_opts = kernel_opts
        self.bias_opts = bias_opts
    
    def kernel_initializer(self, shape, name=None):
        if (self.kernel_opts != None):
            values = np.random.normal(loc=self.kernel_opts['loc'], scale=self.kernel_opts['scale'], size=shape)
            return K.variable(values, name=name)
        else:
            return None
        
    def bias_initializer(self, shape, name=None):
        if (self.bias_opts != None):
            values = np.random.normal(loc=self.bias_opts['loc'], scale=self.bias_opts['scale'], size=shape)
            return K.variable(values, name=name)
        else:
            return None
        
    def get_full_model(self):
        base_model = self.base_model(
            include_top=False, 
            weights=self.opts['weights'], 
            input_shape=self.opts['input_shape'],
            pooling=self.opts['pooling'],
        )
        
        for layer in base_model.layers:
            layer.trainable = False
        
        input_1 = Input(self.opts['input_shape'])
        input_2 = Input(self.opts['input_shape'])

        feture_vec_1 = base_model(input_1)
        feture_vec_2 = base_model(input_2)
        distance = subtract([feture_vec_1, feture_vec_2])

        prediction = Dense(1, activation='sigmoid', bias_initializer=self.bias_initializer)(distance)
        model = Model(input=[input_1, input_2], output=prediction)
        return model
    
    def get_top_model(self):
        input_1 = Input(self.opts['features_shape'])
        input_2 = Input(self.opts['features_shape'])
        
        distance = subtract([input_1, input_2])
        
        prediction = Dense(1, activation='sigmoid', bias_initializer=self.bias_initializer)(distance)
        model = Model(input=[input_1, input_2], output=prediction)
        return model

In [216]:
model = Seamese_Model(
    ResNet50, 
    opts={
        'weights': 'imagenet',
        'input_shape': (img_size, img_size, 3),
        'features_shape': (1, 2048),
        'pooling': 'avg',
    },
    kernel_opts={
        'loc': 0,
        'scale': 1e-2,
    },
    bias_opts={
        'loc': 0.5,
        'scale': 1e-2,
    }
)

model = model.get_top_model()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [217]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1, 2048)      0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 1, 2048)      0                                            
__________________________________________________________________________________________________
subtract_3 (Subtract)           (None, 1, 2048)      0           input_6[0][0]                    
                                                                 input_7[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 1, 1)         2049        subtract_3[0][0]                 
Total para

## Supervisor

In [218]:
class Supervisor:
    # use seed only for testing
    def __init__(self, data, classes, seed=None):
        self.data = data
        self.classes = classes
        self.seed = seed
        
    def get_pair(self, index_1, index_2):
        el_1 = np.take(self.data, [index_1], axis=0)
        el_2 = np.take(self.data, [index_2], axis=0)
        return([el_1, el_2])
    
    def get_selection_index(self, index, indices):
        selection_index = index
        while selection_index == index:
            selection_index = np.random.choice(indices, 1)[0]
        return selection_index
    
    def get_batch(self, n):
        np.random.seed(self.seed)
        indices = np.random.choice(list(range(len(self.data))), size=n)
        
        pairs = []
        y = []
        
        for index in indices[:n//2]:
            selection_indices = np.argwhere(self.classes == self.classes[index]).flatten()
            selection_index = self.get_selection_index(index, selection_indices)
            pairs.append(self.get_pair(index, selection_index))
            y.append(1)
            
        for index in indices[n//2:]:
            selection_indices = np.argwhere(self.classes != self.classes[index]).flatten()
            selection_index = self.get_selection_index(index, selection_indices)
            pairs.append(self.get_pair(index, selection_index))
            y.append(0)
            
        return (np.array(pairs), np.array(y))

In [227]:
features_train = np.load('./data/features/res_net/features_train.npy')
classes_train = np.load('./data/features/res_net/classes_train.npy')

In [220]:
supervisor = Supervisor(features_train, classes_train, 1)

In [222]:
%%time
supervisor.get_batch(128)[0].shape

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 12.3 ms


(128, 2, 1, 2048)